In [1]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import pandas as pd
import requests

In [ ]:
# Coloca tus credenciales aquí
CLIENT_ID = ''
CLIENT_SECRET = ''
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Coloca tus credenciales de lastfm aquí
SHARED_SECRET = ''
API_KEY = ''

last_fm_connect = requests.get(f"http://www.last.fm/api/auth/?api_key={API_KEY}")

In [5]:
paises_mediterraneos = {
    "ES": "España",
    "FR": "Francia",
    "MC": "Mónaco",
    "IT": "Italia",
    "SI": "Eslovenia",
    "HR": "Croacia",
    "BA": "Bosnia y Herzegovina",
    "ME": "Montenegro",
    "AL": "Albania",
    "GR": "Grecia",
    "MT": "Malta",
    "CY": "Chipre"}

In [6]:
def extraccion_por_paises(diccionario_zonas, años):
    todos_los_datos = [] # Esto es una lista de dicts donde se van guardando los diccionarios de cada país
    for pais in diccionario_zonas.keys():
        datos_rango = sp.search(q = f'year:{años}', limit=50, offset=0, type='track', market={pais}) # Cada una tiene que cambiar en q el rango de años
        datos_por_pais = {}
        datos_por_pais['pais'] = pais
        datos_por_pais['datos'] = datos_rango
        todos_los_datos.append(datos_por_pais)
    return todos_los_datos
   

In [7]:
def filtro_de_datos(lista_datos_zonas, datos):
    for datos_pais in lista_datos_zonas:
        for element in datos_pais['datos']['tracks']['items']:
            datos['pais'].append(datos_pais['pais'])
            datos['popularity'].append(element['popularity'])
            datos['nombre'].append(element['name'])
            artistas = []
            for artista in element['artists']:
                artistas.append(artista['name'])   
            artistas_str = ', '.join(artistas)
            datos['artista'].append(artistas_str)

In [8]:
p_mediterraneos = extraccion_por_paises(paises_mediterraneos, '2000-2004')

In [9]:
dic_mediterraneos = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}

In [10]:
filtro_de_datos(p_mediterraneos, dic_mediterraneos)

In [11]:
dic_mediterraneos

{'nombre': ['Soldadito marinero',
  'Angel',
  'Rosas',
  'Dímelo bajito',
  'La vereda de la puerta de atrás',
  'San Pedro',
  'La casa por el tejado',
  'El peligro',
  'Yellow',
  'Eso de saber',
  'Carolina - En directo 2000',
  'Mestizo',
  'Dile',
  'Ni na no',
  'Vino Tinto',
  'He Loves U Not',
  'Cuando zarpa el amor',
  'La vita è',
  'Puedes Contar Conmigo',
  'Stuck in the Middle with You',
  'The Scientist',
  'Cabelo Raspadinho',
  'Stand By',
  '2 Faced',
  'Somewhere Only We Know',
  'This Is Me',
  'Without Me',
  'Dar A Volta No Brasil - Ao Vivo',
  'El aire de la calle',
  'Beautiful Inside',
  'Numb',
  'Quemando tus recuerdos - Gira 2002',
  'Obsesion',
  "American Life (Director's Cut) - 4K Remaster",
  'Quiereme',
  'HEY! JORDU!',
  'La Playa',
  'Welcome To Hell (Videoclip)',
  'Cómo hablar',
  'Devolva-Me',
  'La Madre de Jose',
  'This Is My Time',
  'In the End',
  'Rooftop',
  'Insoportable',
  'Kiedy góral umiera',
  'Clocks',
  'Here She Comes Again',
  '

In [12]:
lista_nombres_mediterraneo = list(dic_mediterraneos.values())[0]
lista_artistas_mediterraneo = list(dic_mediterraneos.values())[2]

In [13]:
def llamada_lastfm(lista_nombres, lista_artistas, datos):
    for i in range(len(lista_nombres)):
        nombre = str(lista_nombres[i]).replace(' ', '+')
        artista = lista_artistas[i].replace(' ', '+').replace(',', '+&')
        llamada_genero = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist={artista}&track={nombre}&api_key={API_KEY}&autocorrect=1&format=json")
        try:
            genero_cancion = llamada_genero.json()['toptags']['tag'][0]['name']
            datos['genero'].append(genero_cancion)
        except KeyError:
            datos['genero'].append("NULL")
            print(f"La canción {lista_nombres[i]} no está registrada en lastfm")
            
        except IndexError:
            datos['genero'].append("NULL")
            print(f"La canción {lista_nombres[i]} no tiene tags")

In [14]:
llamada_lastfm(lista_nombres_mediterraneo, lista_artistas_mediterraneo, dic_mediterraneos)

La canción Angel no está registrada en lastfm
La canción Dímelo bajito no tiene tags
La canción San Pedro no está registrada en lastfm
La canción Carolina - En directo 2000 no tiene tags
La canción Ni na no no tiene tags
La canción Dar A Volta No Brasil - Ao Vivo no tiene tags
La canción Quemando tus recuerdos - Gira 2002 no está registrada en lastfm
La canción Obsesion no está registrada en lastfm
La canción American Life (Director's Cut) - 4K Remaster no tiene tags
La canción HEY! JORDU! no tiene tags
La canción Welcome To Hell (Videoclip) no tiene tags
La canción Angel no está registrada en lastfm
La canción Dímelo bajito no tiene tags
La canción Manhattan-Kaboul no está registrada en lastfm
La canción Dar A Volta No Brasil - Ao Vivo no tiene tags
La canción Yeah! (feat. Lil Jon & Ludacris) no está registrada en lastfm
La canción San Pedro no está registrada en lastfm
La canción Crazy In Love (feat. JAY-Z) no está registrada en lastfm
La canción American Life (Director's Cut) - 4K R

In [15]:
df_mediterraneos = pd.DataFrame(dic_mediterraneos)

In [16]:
df_mediterraneos.index.name="id"

In [17]:
df_mediterraneos

,nombre,popularity,artista,pais,genero
id,,,,,
0,Soldadito marinero,62,Fito y Fitipaldis,ES,spanish
1,Angel,28,"Shaggy, Rayvon",ES,NULL
2,Rosas,75,La Oreja de Van Gogh,ES,pop
3,Dímelo bajito,0,Manzanita,ES,NULL
4,La vereda de la puerta de atrás,60,Extremoduro,ES,rock
...,...,...,...,...,...
595,Love Is Alive - Live; 2004 Remaster,0,Joe Cocker,CY,NULL
596,Chop Suey!,79,System Of A Down,CY,alternative metal
597,Ni na no,0,Javier Álvarez,CY,NULL


In [19]:
df_mediterraneos.to_csv("paises_mediterraneros_2000_2004.csv")